In [3]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [4]:
files = sorted(glob('./data/*.csv'))
data_fit = pd.DataFrame()
for file in tqdm(files[:1]):
    df_day = pd.read_csv(file)
    data_fit = pd.concat([data_fit, df_day['okt']])
data_fit.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.48it/s]


In [5]:
data_fit

,0
0,한국 배구 연맹 23일 주말 열리다 프로배구 4 경기 연기 다 고 1일 밝히다 중계...
1,아동 성범죄자 조두순 69 사진 지난달 추다 뒤 보름 외부 활동 나서다 것 전해지다...
2,서울 신문 추미애 구치소 집단 감염 첫 사과 집단 감염 송구 취약 부분 드러나다 빠...
3,정부 수도권 사회 거리 두기 조 정안 2일 발표 하다 하다 상황 지금 단계 몇 주 ...
4,강릉 뉴스 1 이종재 기자 새해 첫날 1일 강원 강릉 지역 찾다 해 맞이 객 지난해...
...,...
1207,정읍 뉴시스 김얼 기자 2021년 신축년 희다 소 띠 해 상서 기운 물씬 일어나다 ...
1208,정읍 뉴시스 김얼 기자 2021년 신축년 희다 소 띠 해 상서 기운 물씬 일어나다 ...
1209,서울 뉴시스 홍세 희다 기자 최대 집 대한의사협회 1일 새해 94 의정 합의 정신 ...
1210,서울 뉴시스 구무 기자 김용익 건강 보험 공단 이사장 오다 한해 건강 보험 보장 률...


# 모델 학습, 저장

In [6]:
from sentence_transformers import SentenceTransformer
import pickle

In [7]:
model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')

In [8]:
arc_content = """2011년 한국을 떠나 러시아로 귀화한 전 쇼트트랙 선수 빅토르 안(38·한국명 안현수)의 국내 복귀 시도가 일단 무산됐다. 경기도 성남시청 직장운동부 빙상팀 코치에 지원했으나, 최종 후보에 들지 못해서다.

성남시는 29일 “시청 빙상팀 코치직 채용 전형에 빅토르 안을 포함해 7명이 지원했는데, 빅토르 안은 상위 2배수 후보에 들지 않았다”고 밝혔다. 시 관계자는 “서류와 면접 심사를 통해 기술, 소통 능력 등 여러 요소를 종합해 판단했다”며 “빙상계 여론과 언론 보도 등을 통해 나오는 시각도 평가에 반영됐다”고 했다.

성남시는 지난해 12월 19일 빙상팀 코치를 뽑기 위한 채용 공고를 냈다. 빅토르 안과 베이징 동계올림픽에서 중국 대표팀을 이끈 김선태 전 감독 등 7명이 지원했다. 김 전 감독도 최종 후보에 들지 못한 것으로 전해졌다.

빅토르 안은 2006년 토리노 동계올림픽에서 3개의 금메달을 딴 한국 쇼트트랙의 간판이었다. 2011년 소속팀이었던 성남시청이 빙상팀을 해체하자 선수 생활을 이어가기 위해 러시아로 귀화했다. 2014년 소치 동계올림픽에는 러시아 선수로 나서 3관왕에 올랐다. 2018년 평창 동계올림픽 출전이 무산된 이후에는 은퇴를 선언하고 지도자로 변신했다. 2022년 베이징 동계올림픽에서는 중국 대표팀 코치로 활동했다.

성남시는 31일 빙상팀 코치 선발 결과를 발표할 예정이다."""

In [20]:
%%time
embeddings = model.encode(data_fit.loc[:100,0], show_progress_bar=True, convert_to_tensor=True).tolist()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 1min 39s
Wall time: 25.3 s


In [24]:
%%time
embeddings2 = model.encode(data_fit.loc[:100,0], show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 1min 41s
Wall time: 25.7 s


In [21]:
%%time
embeded_content = model.encode([arc_content], convert_to_tensor=True).tolist()

CPU times: total: 953 ms
Wall time: 256 ms


In [25]:
%%time
embeded_content2 = model.encode([arc_content], convert_to_tensor=True)

CPU times: total: 906 ms
Wall time: 235 ms


# torch

In [9]:
from torch.nn import CosineSimilarity

In [114]:
cos = CosineSimilarity(dim=1)

In [144]:
%%time
cos_sim_res1 = cos(embeded_content, embeddings)

CPU times: total: 0 ns
Wall time: 6.12 ms


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
%%time
cos_sim_res2 = cosine_similarity(embeded_content, embeddings)

CPU times: total: 15.6 ms
Wall time: 35.5 ms


# sentence_transformers

In [26]:
from sentence_transformers import util

In [27]:
%%time
cos_sim_res3 = util.cos_sim(embeded_content, embeddings)

CPU times: total: 15.6 ms
Wall time: 16.6 ms


In [130]:
cos_sim_res3[0]

tensor([0.2455, 0.1644, 0.1944,  ..., 0.2164, 0.1999, 0.1721])

In [125]:
len(cos_sim_res3[0])

1212

# tfidf, sklearn cos_sim

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2), min_df=1)

In [31]:
%%time
tfidf_dtm = tfidf_vec.fit_transform(data_fit.loc[:100,0])

CPU times: total: 93.8 ms
Wall time: 116 ms


In [32]:
%%time
tfidf_content = tfidf_vec.transform([arc_content])

CPU times: total: 0 ns
Wall time: 1 ms


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
%%time
cos_sim_res2 = cosine_similarity(tfidf_content, tfidf_dtm)

CPU times: total: 0 ns
Wall time: 5.16 ms


## 기본 함수

In [23]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [24]:
def string_tokenizer(content):
    import pandas as pd
    from konlpy.tag import Okt
    okt = Okt()
    
    content = pd.Series(content)
    # 한글, 숫자, 공백만 남기기
    content = content.str.replace('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)
    token_content = okt.pos(content[0], norm=True, stem=True)
    return token_content

In [25]:
except_list = ['Exclamation', 'Josa', 'KoreanParticle', 'Determiner',  'Eomi', 'Suffix',  'VerbPrefix', 'PreEomi']
include_list = ['Verb',  'Noun']

In [26]:
# 230207 기현
def tag_except_list(except_list, token_content):
    filtered_list= []
    for tag in token_content :
        if tag[1] in except_list:
            continue
        filtered_list.append(tag[0])
    content = " ".join(filtered_list)
    return content

In [27]:
# 230207 기현
def tag_include_list(include_list, token_content):
    filtered_list=[]
    for tag in token_content:
        if tag[1] in include_list:
            filtered_list.append(tag[0])

    content = " ".join(filtered_list)
    return content

# torch_cos_sim, sent_transformer

In [133]:
def get_recommend_by_sent(content, n, drange=False):
    from torch.nn import CosineSimilarity
    cos = CosineSimilarity(dim=1)
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    embeded_content = model.encode([token], convert_to_tensor=True)
    
    cos_sim_res1 = cos(embeded_content, embeddings)
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res1))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_fit.iloc[article_idx][[0]]
    # rst['similarity'] = [sim[1] for sim in sim_scores_n]
    rst['similarity'] = [round(sim[1].item(), 4)*100 for sim in sim_scores_n]
        
    return rst

In [134]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_sent(arc_content, 10)

CPU times: total: 438 ms
Wall time: 117 ms


,0,similarity
312,창원 뉴시스 김기진 기자 정의당 경 남도 신축년 새해 맞다 19일 째를 맞이 중대 ...,44.07
1001,서울 뉴스 1 심 언 기 기자 기부 어떻다 마약 보다 더하다 중독 있다 하다 기부 ...,44.02
740,서울 연합뉴스 이재윤 기자 국내 신종 코로나바이러스 감염증 코로나 19 확산 세 지...,42.47
753,서울 연합뉴스 신 선미 기자 국내 신종 코로나바이러스 감염증 코로나 19 확산 세 ...,41.02
1000,서울 뉴스 1 심 언 기 기자 연말 연 시 길거리 오가다 이 옷깃 가방 사랑 열매 ...,39.89
674,서울 연합뉴스 신 선미 기자 국내 신종 코로나바이러스 감염증 코로나 19 확 진자 ...,38.63
492,서울 뉴스 1 이형진 기자 이영 기자 2021년 새해 신종 코로나바이러스 감염증 코...,38.41
873,병역 기피 의혹 대한민국 입국 금지 처분 받다 가수 유승준 미국 스티브 유 44 이...,38.11
363,광주 뉴스 1 고귀하다 기자 광주 동구 지산 2 동 은 주민 마을 지키다 두 그루 ...,37.93
567,서울 뉴시스 하종 민 기자 2021년 새해 첫날 서울 신종 코로나바이러스 감염증 코...,37.84


# sent_transformers

In [127]:
def get_recommend_by_sentt(content, n, drange=False):
    from sentence_transformers import util
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    embeded_content = model.encode([token], convert_to_tensor=True)
    
    cos_sim_res3 = util.cos_sim(embeded_content, embeddings)
    
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res3[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_fit.iloc[article_idx][[0]]
    # rst['similarity'] = [sim[1] for sim in sim_scores_n]
    rst['similarity'] = [round(sim[1].item(), 4)*100 for sim in sim_scores_n]
        
    return rst

In [128]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_sentt(arc_content, 10)

CPU times: total: 547 ms
Wall time: 138 ms


,0,similarity
312,창원 뉴시스 김기진 기자 정의당 경 남도 신축년 새해 맞다 19일 째를 맞이 중대 ...,44.07
1001,서울 뉴스 1 심 언 기 기자 기부 어떻다 마약 보다 더하다 중독 있다 하다 기부 ...,44.02
740,서울 연합뉴스 이재윤 기자 국내 신종 코로나바이러스 감염증 코로나 19 확산 세 지...,42.47
753,서울 연합뉴스 신 선미 기자 국내 신종 코로나바이러스 감염증 코로나 19 확산 세 ...,41.02
1000,서울 뉴스 1 심 언 기 기자 연말 연 시 길거리 오가다 이 옷깃 가방 사랑 열매 ...,39.89
674,서울 연합뉴스 신 선미 기자 국내 신종 코로나바이러스 감염증 코로나 19 확 진자 ...,38.63
492,서울 뉴스 1 이형진 기자 이영 기자 2021년 새해 신종 코로나바이러스 감염증 코...,38.41
873,병역 기피 의혹 대한민국 입국 금지 처분 받다 가수 유승준 미국 스티브 유 44 이...,38.11
363,광주 뉴스 1 고귀하다 기자 광주 동구 지산 2 동 은 주민 마을 지키다 두 그루 ...,37.93
567,서울 뉴시스 하종 민 기자 2021년 새해 첫날 서울 신종 코로나바이러스 감염증 코...,37.84


# skearn_cos_sim, tfidf

In [89]:
def get_recommend_by_tfidf(content, n, drange=False):
    from sklearn.metrics.pairwise import cosine_similarity
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    tfidf_content = tfidf_vec.transform([token])
    
    cos_sim_res2 = cosine_similarity(tfidf_content, tfidf_dtm)
    print(cos_sim_res2[0])
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res2[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_fit.iloc[article_idx][[0]]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
        
    return rst

In [90]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_tfidf(arc_content, 10)

[0.00349746 0.00920457 0.0133762  ... 0.00797768 0.01787666 0.01167992]
CPU times: total: 46.9 ms
Wall time: 28.7 ms


,0,similarity
1001,서울 뉴스 1 심 언 기 기자 기부 어떻다 마약 보다 더하다 중독 있다 하다 기부 ...,10.54
843,편집 자주 차다 고단 지난 하다 하다 신종 코로나바이러스 감염증 코로나 19 지금껏...,6.73
1049,경향신문 문재인 정부 3년 6 개월 민주주의 둘러싸다 실험 하다 문 대통령 촛불집회...,5.31
925,뉴스 스크립트 더불다 민주당 이낙연 대표 적절하다 시기 두 전직 대통령 사면 문재인...,5.03
1186,이대 규 24 씨 마이스터고 졸업 뒤 대학 진학 하다 않다 곧바로 일 시작 하다 기...,4.92
981,서울 뉴스 1 김윤경 기자 하다 해 끝나다 새해 시작 되다 보통 사람 라면 자성 하...,4.87
1048,경향신문 지난해 전 세계 덮다 친 코로나 19 를 계기 기본소득 논쟁 본격 돼다 정...,4.48
57,코로나 19 바이러스 사회 구성원 무차별 공격 하다 누군가 더 가혹하다 코로나 19...,4.33
930,서울 뉴스 1 온 다예 기 자강 수련 기자 김근 욱 기자 2021년 신축년 새해 밝...,4.10
1044,경향신문 2020년 하다 해 할퀴다 간 코로나 19 팬데믹 세계 대유행 한국 사회 ...,3.86
